In [160]:
import numpy as np 
import pandas as pd 

In [161]:
games = pd.read_csv('games.csv')
games.shape

(1512, 14)

In [162]:
games.head(1)

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K


In [163]:
#team
#rating
#genres
#title
#summary
#reviews

games = games[['Title','Team','Genres','Rating','Summary','Reviews']]

In [164]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Title    1512 non-null   object 
 1   Team     1511 non-null   object 
 2   Genres   1512 non-null   object 
 3   Rating   1499 non-null   float64
 4   Summary  1511 non-null   object 
 5   Reviews  1512 non-null   object 
dtypes: float64(1), object(5)
memory usage: 71.0+ KB


In [165]:
games.head(1512)

,Title,Team,Genres,Rating,Summary,Reviews
0,Elden Ring,"['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5,"Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o..."
1,Hades,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...
2,The Legend of Zelda: Breath of the Wild,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4,The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...
3,Undertale,"['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,"A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...
4,Hollow Knight,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4,A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with..."
...,...,...,...,...,...,...
1507,Back to the Future: The Game,['Telltale Games'],"['Adventure', 'Point-and-Click']",3.2,Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...
1508,Team Sonic Racing,"['Sumo Digital', 'Sega']","['Arcade', 'Racing']",2.9,Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C..."
1509,Dragon's Dogma,['Capcom'],"['Brawler', 'RPG']",3.7,"Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ..."
1510,Baldur's Gate 3,['Larian Studios'],"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",4.1,"An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...


In [166]:
games.isnull().sum()

Title       0
Team        1
Genres      0
Rating     13
Summary     1
Reviews     0
dtype: int64

In [167]:
games.dropna(inplace=True)

In [168]:
games.duplicated().sum()

np.int64(384)

In [169]:
games.drop_duplicates()

,Title,Team,Genres,Rating,Summary,Reviews
0,Elden Ring,"['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5,"Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o..."
1,Hades,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...
2,The Legend of Zelda: Breath of the Wild,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4,The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...
3,Undertale,"['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,"A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...
4,Hollow Knight,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4,A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with..."
...,...,...,...,...,...,...
1507,Back to the Future: The Game,['Telltale Games'],"['Adventure', 'Point-and-Click']",3.2,Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...
1508,Team Sonic Racing,"['Sumo Digital', 'Sega']","['Arcade', 'Racing']",2.9,Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C..."
1509,Dragon's Dogma,['Capcom'],"['Brawler', 'RPG']",3.7,"Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ..."
1510,Baldur's Gate 3,['Larian Studios'],"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",4.1,"An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...


In [175]:
games['Tags']=games['Team']+games['Genres']+games['Summary']+games['Reviews']

In [176]:
games.head()

,Title,Team,Genres,Rating,Summary,Reviews,Tags
0,Elden Ring,"['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5,"Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...","['Bandai Namco Entertainment', 'FromSoftware']..."
1,Hades,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,"['Supergiant Games']['Adventure', 'Brawler', '..."
2,The Legend of Zelda: Breath of the Wild,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4,The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,"['Nintendo', 'Nintendo EPD Production Group No..."
3,Undertale,"['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,"A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,"['tobyfox', '8-4']['Adventure', 'Indie', 'RPG'..."
4,Hollow Knight,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4,A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...","['Team Cherry']['Adventure', 'Indie', 'Platfor..."


In [177]:
new_df=games[['Title','Rating','Tags']]

In [179]:
new_df['Tags'][0]

<class 'pandas.core.frame.DataFrame'>
Index: 1498 entries, 0 to 1511
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   1498 non-null   object 
 1   Rating  1498 non-null   float64
 2   Tags    1498 non-null   object 
dtypes: float64(1), object(2)
memory usage: 79.1+ KB
